# Topic Modeling

El modelamiento de tópicos, es una herramienta estadística que busca encontrar los temas presentes en un conjunto de documentos (corpus), permitiendo organizar, buscar, indexar, explorar y comprender grandes colecciones de documentos.\\


En este sentido, los temas se pueden definir como ``un patrón repetitivo de términos co-currentes en un corpus''. Por ejemplo, se tiene el siguiente tópico, representado por sus cuatro palabras más probables, ``salud'', ``médico'', ``paciente'', ``hospital'', estas palabras sugieren el siguiente nombre para el tema: ``Atención médica‘’.En los modelos convencionales de tópicos:
1. Las palabras dentro de un documento y los documentos son tratados como intercambiables.
2. La mayoría de estos fijan el número de tópicos y lo  mantienen así a lo largo del corpus completo.



### VISUALIZACIÓN DE QUE ES UN TÓPICO

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import operator
import itertools
import re
from numpy.linalg import norm
from time import time

#Preprocesamiento
import spacy
from spacy.lang.es.stop_words import STOP_WORDS #importar set de stopwords
from nltk.stem import SnowballStemmer #importar stemmer
nlp = spacy.load('es_core_news_sm') #python -m spacy download es

#Bag-of-words
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.feature_extraction.text import TfidfVectorizer

#Topic modeling

from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamodel import LdaModel
import pyLDAvis
import pyLDAvis.gensim

### Base de datos
La base de datos consiste de 5 conjuntos de noticias extraídas de la radio biobio. 
Cada conjunto de noticias contiene 200 documentos (noticias) y tiene asociado una categoría en {nacional, internacional, economía, sociedad, opinion}.

**Creditos**: 

- Pablo Badilla Torrealba 
- https://github.com/dccuchile/CC6205/tree/master/tutorials

In [ ]:
df = pd.read_csv('news_biobio.csv', index_col=0)

In [3]:
def tokenizer(doc, sep=None, stopwords_remove =False, lemmatization=False, stemming = False, vocabulary=None):
    '''
    Por defecto divide la sentencia por el carácter espacio.
    Ej: 'Data Mining is the best course'->['Data',  'Mining', 'is', 'the', 'best', 'course']
    
    Input: 
    1. doc: str, documento.
    2. sep: str, carácter para dividir el documento en tokens, por defecto es el espacio.
    3. stopwords_remove: bool, si es True remueve los stopwords del documento.
    4. lemmatization: bool, si es True lleva las palabras a su lema.
    5. stemming: bool, si es True lleva las palabas a su raíz.
    6. vocabuary: list, si un vocabulario es dado filtra las palabras que no estan presentes en el.
    
    Output: 
    list, lista de tokens.
    
    Nota: aplicar stemming y lemmatization al mismo tiempo no es correcto.
    '''
    doc = re.sub(r'[^\w\s]','', doc) #elimina los símbolos de puntuación
    doc = re.sub(r'[a-zA-Z]+[0-9]+', '', doc) #elimina los caracteres que contienen letras y números
    doc = re.sub(r'[0-9]+', ' ', doc) #elimina los caracteres numéricos
   
    tokens = doc.split(sep) #tokenización
    tokens = [word.lower() for word in tokens] #pasar todas las palabras a minúsculas
    
    
    
    if stopwords_remove ==True: #remover stopwords y palabras con menos de tres caracteres
        tokens = [word for word in tokens if word not in STOP_WORDS and len(word)>2]
    
    if lemmatization==True:
        tokens = [nlp(word)[0].lemma_ for word in tokens]
        
    if stemming == True:
        stemmer = SnowballStemmer('spanish')
        tokens = [stemmer.stem(word) for word in tokens]
    
    if vocabulary is not None:
        tokens = [word for word in tokens if word in vocabulary]
    
    return tokens


## 1. Latent Dirichlet Allocation
 
Sean  *K* tópicos, $\beta_{1:K}$ son distribuciones de probabilidad sobre un vocabulario fijo, dibujadas por una $Dirichlet(\eta)$. Para cada documento $d$ del corpus $D$ se asume que es dibujado por el siguiente proceso generativo:


1. Escoger la mezcla de tópicos $\theta_{d}$ de una distribución sobre un $(K− 1)-simplex$, tal como una $Dirichlet(\alpha)$.
2. Para cada palabra:
    - Escoger la asignación del tópico $z\sim Mult(\theta_{d})$
    - Escoger una palabra $w \sim Mult(\beta_{z})$


<img src='img/lda_graphical_model.png'>

 
### Ejemplo de output
### Revisar Murphy Mixture Models
### K-means
 
Blei, D. M., Ng, A. Y., and Jordan, M. I. (2003). Latent dirichlet allocation. Journal of machine Learning research, 3(Jan):993–1022.

In [ ]:
#Creamos el diccionario a partir de los textos procesados en el formato que necesita LDA en gensim
dictionary = Dictionary(newcorpus)

#Creamos el corpus para darle al modelo (segun el formato de esta libreria)
#El corpus contiene una representacion numerica de los textos, un texto es representada por una lista de tuplas
#donde el primer elemento de la tupla es la id de la palabra y el segundo es su frecuencia de aparición en el texto.


corpus = [dictionary.doc2bow(text) for text in newcorpus]

#guardamos el diccionario y el corpus
dictionary.save('dictionary.dict')
MmCorpus.serialize('corpus.mm', corpus)

In [ ]:
print('Primer elemento del diccionario o bolsa de palabras:', dictionary[0])
print('Representación del corpus en el formato que requiere la librería: \n', corpus[0:2] )

In [ ]:
Lda_k_5=LdaModel(corpus=corpus, id2word=dictionary, num_topics=5) 
Lda_k_5.save('Lda_k_5.model') # guardamos el modelo

## Visualización LDA

Las visualizaciones en modelamiento de tópicos nos ayudan a responder tres preguntas:

1. **¿Cuál es el significado de cada tópico?**
2. **¿Cuán predominante es cada tópico?**
3. **¿Cómo se relacionan los tópicos entre sí?**



Sievert, C., & Shirley, K. (2014), desarrollaron una herramienta de visualización para responder estas preguntas. La herramienta a través de una visualización espacial responde la pregunta 2 y 3.  

Además para responder la pregunta 1 incorporan un gráfico de barras a la derecha del gráfico espacial que muestra las palabras más relevantes del tópico seleccionado dado un parámetro $\lambda$ entre 0 y 1,  entonces, la relevancia de la palabra w en el tópico $k$ dado $\lambda$ esta dada a  través de la siguiente formula:
\begin{equation*}
r(w,k|\lambda) = \lambda log(\phi_{k,w})+(1-\lambda)log(\frac{\phi_{k,w}}{p_{w}}), \lambda \in [0,1]$$
\end{equation*}
Donde $\phi_{k,w}$ es la probabilidad de que el término $w$ sea generado por el tópico $k$, $p_{w}$ es la probabilidad de el término $w$ en el corpus.\\  


\footnote{Sievert, C., & Shirley, K. (2014). LDAvis: A method for visualizing and interpreting topics. In Proceedings of the workshop on interactive language learning, visualization, and interfaces (pp. 63-70).} 

\href{http://nbviewer.jupyter.org/github/bmabey/hacker_news_topic_modelling/blob/master/HN\%20Topic\%20Model\%20Talk.ipynb#topic=3&lambda=0.46&term=}{ \texttt{Click aquí: LDAvis example}}
  

In [ ]:
#Preparar los datos para generarar la visualización de LDA
lda_display = pyLDAvis.gensim.prepare(Lda_k_5, corpus, dictionary, sort_topics=True, R=30)